In [30]:
#pip install pandas OpenBlender scikit-learn
#pip install tensorflow

In [31]:
import warnings
warnings.filterwarnings('ignore')
import nltk
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import OpenBlender
import json

In [32]:
token = '621bd93e9516291fadccaea5AVFN4rSSNmyghGLpu6u9NBmNRphWDV'
action = 'API_getObservationsFromDataset'
parameters = { 
    'token' : token,
    'id_dataset' : '5d4c3af79516290b01c83f51',
    'sample_size':2811,   
}
df_bitcoin = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df_bitcoin.reset_index(drop=True, inplace=True)
df_bitcoin['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df_bitcoin.timestamp]
df_bitcoin = df_bitcoin.drop('timestamp', axis = 1)
df_bitcoin

Task ID: '621e79130895fafb4a9d8d4b'.
Total estimated consumption: 500.28 processing units.
20.0 %
40.0 %
60.0 %
80.0 %
100.0 % completed.


,volume,price,high,low,open,change,date
0,2.75K,49264.0,49487.0,48121.0,48868.0,0.81%,22-08-2021 17:00:00
1,21.16K,49747.0,51154.0,48120.6,49725.0,0.04%,13-05-2021 17:00:00
2,14.96K,49725.0,57688.0,49725.0,56715.0,-12.32%,12-05-2021 17:00:00
3,7.69K,56715.0,56724.0,54789.0,55851.0,1.55%,11-05-2021 17:00:00
4,13.49K,55851.0,59454.3,55448.0,58276.0,-4.16%,10-05-2021 17:00:00
...,...,...,...,...,...,...,...
2806,-,13.4,13.4,13.4,13.4,-0.44%,05-01-2013 16:00:00
2807,-,13.5,13.5,13.5,13.5,0.75%,04-01-2013 16:00:00
2808,-,13.4,13.4,13.4,13.4,0.90%,03-01-2013 16:00:00
2809,-,13.3,13.3,13.3,13.3,0.00%,02-01-2013 16:00:00


In [33]:
token = '6219235f9516292d6a055228wa5Vo6VaFIreOf50OUDBm7oTf515s3'
action = 'API_getObservationsFromDataset'
parameters = { 
    'token' : token,
    'id_dataset' : '5d4c83f79516290b01c8fe6e',
    'sample_size':1825
}
df_ether = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df_ether.reset_index(drop=True, inplace=True)
df_ether['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df_ether.timestamp]
df_ether = df_ether.drop('timestamp', axis = 1)
df_ether

Task ID: '621e79200895fafb4a9d8d4b'.
Total estimated consumption: 500.18 processing units.
25.0 %
50.0 %
75.0 %
100.0 % completed.


,volume,price,high,low,open,change,date
0,733.69K,2922.35,2949.39,2573.85,2615.01,11.70%,28-02-2022 17:00:00
1,650.22K,2616.15,2834.24,2563.00,2779.56,-5.77%,27-02-2022 17:00:00
2,418.49K,2776.37,2856.11,2734.80,2768.64,0.33%,26-02-2022 17:00:00
3,679.41K,2767.33,2829.99,2573.23,2599.00,6.59%,25-02-2022 17:00:00
4,1.40M,2596.16,2734.17,2302.82,2578.87,0.70%,24-02-2022 17:00:00
...,...,...,...,...,...,...,...
1820,26.34K,12.50,15.07,11.40,15.07,-17.05%,14-03-2016 16:00:00
1821,0.37K,15.07,15.07,12.92,12.92,16.64%,13-03-2016 16:00:00
1822,0.24K,12.92,13.45,11.95,11.95,8.12%,12-03-2016 16:00:00
1823,0.05K,11.95,11.95,11.75,11.75,1.70%,11-03-2016 16:00:00


In [34]:
df_bitcoin_init = pd.DataFrame()
df_ether_init= pd.DataFrame()

In [35]:
df_bitcoin_init = df_bitcoin

In [36]:
df_bitcoin_init

,volume,price,high,low,open,change,date
0,2.75K,49264.0,49487.0,48121.0,48868.0,0.81%,22-08-2021 17:00:00
1,21.16K,49747.0,51154.0,48120.6,49725.0,0.04%,13-05-2021 17:00:00
2,14.96K,49725.0,57688.0,49725.0,56715.0,-12.32%,12-05-2021 17:00:00
3,7.69K,56715.0,56724.0,54789.0,55851.0,1.55%,11-05-2021 17:00:00
4,13.49K,55851.0,59454.3,55448.0,58276.0,-4.16%,10-05-2021 17:00:00
...,...,...,...,...,...,...,...
2806,-,13.4,13.4,13.4,13.4,-0.44%,05-01-2013 16:00:00
2807,-,13.5,13.5,13.5,13.5,0.75%,04-01-2013 16:00:00
2808,-,13.4,13.4,13.4,13.4,0.90%,03-01-2013 16:00:00
2809,-,13.3,13.3,13.3,13.3,0.00%,02-01-2013 16:00:00


In [37]:
df_ether_init = df_ether

In [38]:
df_ether_init

,volume,price,high,low,open,change,date
0,733.69K,2922.35,2949.39,2573.85,2615.01,11.70%,28-02-2022 17:00:00
1,650.22K,2616.15,2834.24,2563.00,2779.56,-5.77%,27-02-2022 17:00:00
2,418.49K,2776.37,2856.11,2734.80,2768.64,0.33%,26-02-2022 17:00:00
3,679.41K,2767.33,2829.99,2573.23,2599.00,6.59%,25-02-2022 17:00:00
4,1.40M,2596.16,2734.17,2302.82,2578.87,0.70%,24-02-2022 17:00:00
...,...,...,...,...,...,...,...
1820,26.34K,12.50,15.07,11.40,15.07,-17.05%,14-03-2016 16:00:00
1821,0.37K,15.07,15.07,12.92,12.92,16.64%,13-03-2016 16:00:00
1822,0.24K,12.92,13.45,11.95,11.95,8.12%,12-03-2016 16:00:00
1823,0.05K,11.95,11.95,11.75,11.75,1.70%,11-03-2016 16:00:00


In [39]:
df_bitcoin_init.to_csv("Bitcoin.csv",index = False)

In [40]:
df_ether_init.to_csv("Ethereum.csv", index = False)

In [41]:
df_bitcoin_polar=df_bitcoin_init
df_bitcoin_polar['log_diff'] = np.log(df_bitcoin_polar['price']) - np.log(df_bitcoin_polar['open'])
df_bitcoin_polar

,volume,price,high,low,open,change,date,log_diff
0,2.75K,49264.0,49487.0,48121.0,48868.0,0.81%,22-08-2021 17:00:00,0.008071
1,21.16K,49747.0,51154.0,48120.6,49725.0,0.04%,13-05-2021 17:00:00,0.000442
2,14.96K,49725.0,57688.0,49725.0,56715.0,-12.32%,12-05-2021 17:00:00,-0.131531
3,7.69K,56715.0,56724.0,54789.0,55851.0,1.55%,11-05-2021 17:00:00,0.015351
4,13.49K,55851.0,59454.3,55448.0,58276.0,-4.16%,10-05-2021 17:00:00,-0.042503
...,...,...,...,...,...,...,...,...
2806,-,13.4,13.4,13.4,13.4,-0.44%,05-01-2013 16:00:00,0.000000
2807,-,13.5,13.5,13.5,13.5,0.75%,04-01-2013 16:00:00,0.000000
2808,-,13.4,13.4,13.4,13.4,0.90%,03-01-2013 16:00:00,0.000000
2809,-,13.3,13.3,13.3,13.3,0.00%,02-01-2013 16:00:00,0.000000


In [42]:
df_ether_polar=df_ether_init
df_ether_polar['log_diff'] = np.log(df_ether_polar['price']) - np.log(df_ether_polar['open'])
df_ether_polar

,volume,price,high,low,open,change,date,log_diff
0,733.69K,2922.35,2949.39,2573.85,2615.01,11.70%,28-02-2022 17:00:00,0.111120
1,650.22K,2616.15,2834.24,2563.00,2779.56,-5.77%,27-02-2022 17:00:00,-0.060589
2,418.49K,2776.37,2856.11,2734.80,2768.64,0.33%,26-02-2022 17:00:00,0.002788
3,679.41K,2767.33,2829.99,2573.23,2599.00,6.59%,25-02-2022 17:00:00,0.062756
4,1.40M,2596.16,2734.17,2302.82,2578.87,0.70%,24-02-2022 17:00:00,0.006682
...,...,...,...,...,...,...,...,...
1820,26.34K,12.50,15.07,11.40,15.07,-17.05%,14-03-2016 16:00:00,-0.186977
1821,0.37K,15.07,15.07,12.92,12.92,16.64%,13-03-2016 16:00:00,0.153930
1822,0.24K,12.92,13.45,11.95,11.95,8.12%,12-03-2016 16:00:00,0.078045
1823,0.05K,11.95,11.95,11.75,11.75,1.70%,11-03-2016 16:00:00,0.016878


In [43]:
df_bitcoin_polar['target'] = [1 if log_diff > 0 else 0 for log_diff in df_bitcoin_polar['log_diff']]
df_bitcoin_polar

,volume,price,high,low,open,change,date,log_diff,target
0,2.75K,49264.0,49487.0,48121.0,48868.0,0.81%,22-08-2021 17:00:00,0.008071,1
1,21.16K,49747.0,51154.0,48120.6,49725.0,0.04%,13-05-2021 17:00:00,0.000442,1
2,14.96K,49725.0,57688.0,49725.0,56715.0,-12.32%,12-05-2021 17:00:00,-0.131531,0
3,7.69K,56715.0,56724.0,54789.0,55851.0,1.55%,11-05-2021 17:00:00,0.015351,1
4,13.49K,55851.0,59454.3,55448.0,58276.0,-4.16%,10-05-2021 17:00:00,-0.042503,0
...,...,...,...,...,...,...,...,...,...
2806,-,13.4,13.4,13.4,13.4,-0.44%,05-01-2013 16:00:00,0.000000,0
2807,-,13.5,13.5,13.5,13.5,0.75%,04-01-2013 16:00:00,0.000000,0
2808,-,13.4,13.4,13.4,13.4,0.90%,03-01-2013 16:00:00,0.000000,0
2809,-,13.3,13.3,13.3,13.3,0.00%,02-01-2013 16:00:00,0.000000,0


In [44]:
df_ether_polar['target'] = [1 if log_diff > 0 else 0 for log_diff in df_ether_polar['log_diff']]
df_ether_polar

,volume,price,high,low,open,change,date,log_diff,target
0,733.69K,2922.35,2949.39,2573.85,2615.01,11.70%,28-02-2022 17:00:00,0.111120,1
1,650.22K,2616.15,2834.24,2563.00,2779.56,-5.77%,27-02-2022 17:00:00,-0.060589,0
2,418.49K,2776.37,2856.11,2734.80,2768.64,0.33%,26-02-2022 17:00:00,0.002788,1
3,679.41K,2767.33,2829.99,2573.23,2599.00,6.59%,25-02-2022 17:00:00,0.062756,1
4,1.40M,2596.16,2734.17,2302.82,2578.87,0.70%,24-02-2022 17:00:00,0.006682,1
...,...,...,...,...,...,...,...,...,...
1820,26.34K,12.50,15.07,11.40,15.07,-17.05%,14-03-2016 16:00:00,-0.186977,0
1821,0.37K,15.07,15.07,12.92,12.92,16.64%,13-03-2016 16:00:00,0.153930,1
1822,0.24K,12.92,13.45,11.95,11.95,8.12%,12-03-2016 16:00:00,0.078045,1
1823,0.05K,11.95,11.95,11.75,11.75,1.70%,11-03-2016 16:00:00,0.016878,1


In [45]:
format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df_bitcoin_polar['u_timestamp'] = OpenBlender.dateToUnix(df_bitcoin_polar['date'], 
                                           date_format = format, 
                                           timezone = timezone)
df_bitcoin_polar = df_bitcoin_polar[['date', 'u_timestamp', 'price', 'target']]
df_bitcoin_polar.head()

,date,u_timestamp,price,target
0,22-08-2021 17:00:00,1.629652e+09,49264.0,1
1,13-05-2021 17:00:00,1.620925e+09,49747.0,1
2,12-05-2021 17:00:00,1.620839e+09,49725.0,0
3,11-05-2021 17:00:00,1.620752e+09,56715.0,1
4,10-05-2021 17:00:00,1.620666e+09,55851.0,0


In [46]:
format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df_ether_polar['u_timestamp'] = OpenBlender.dateToUnix(df_ether_polar['date'], 
                                           date_format = format, 
                                           timezone = timezone)
df_ether_polar= df_ether_polar[['date', 'u_timestamp', 'price', 'target']]
df_ether_polar.head()

,date,u_timestamp,price,target
0,28-02-2022 17:00:00,1.646068e+09,2922.35,1
1,27-02-2022 17:00:00,1.645981e+09,2616.15,0
2,26-02-2022 17:00:00,1.645895e+09,2776.37,1
3,25-02-2022 17:00:00,1.645808e+09,2767.33,1
4,24-02-2022 17:00:00,1.645722e+09,2596.16,1


In [47]:
search_keyword = 'Crypto'
df_bitcoin_polar = df_bitcoin_polar.sort_values('u_timestamp').reset_index(drop = True)
print('From : ' + OpenBlender.unixToDate(min(df_bitcoin_polar.u_timestamp)))
print('Until: ' + OpenBlender.unixToDate(max(df_bitcoin_polar.u_timestamp)))
OpenBlender.searchTimeBlends(token,
                            df_bitcoin_polar.u_timestamp,
                             search_keyword)

From : 01-01-2013 16:00:00
Until: 22-08-2021 17:00:00


[{'name': 'CryptoCurrency Tweet',
  'url': 'https://www.openblender.io/#/dataset/explore/5ea20b7f9516293633715928',
  'id_dataset': '5ea20b7f9516293633715928',
  'features': ['associated_tweet',
   'author',
   'author_id',
   'favorite_count',
   'hashtags',
   'id',
   'links',
   'mentions',
   're_tweeter',
   'reply_count',
   'retweet_count',
   'text',
   'time',
   'type'],
  'num_observations': 182,
  'intersection': 50.36238515955503,
  'description': 'The latest Twitter threads from CryptoCurrency (@cryptocurr_ncy). With an Ivy League pedigree Andy Clay.'},
 {'name': 'Cardano Price',
  'url': 'https://www.openblender.io/#/dataset/explore/5db1f96a9516291ccd18ad4f',
  'id_dataset': '5db1f96a9516291ccd18ad4f',
  'features': ['change',
   'high',
   'low',
   'open',
   'price',
   'timestamp',
   'volume'],
  'num_observations': 1472,
  'intersection': 42.1547523144174,
  'description': 'Daily Price of Cardano'},
 {'name': 'BTC (crypto) Fear',
  'url': 'https://www.openblender.

In [48]:
search_keyword = 'Crypto'
df_ether_polar = df_ether_polar.sort_values('u_timestamp').reset_index(drop = True)
print('From : ' + OpenBlender.unixToDate(min(df_ether_polar.u_timestamp)))
print('Until: ' + OpenBlender.unixToDate(max(df_ether_polar.u_timestamp)))
OpenBlender.searchTimeBlends(token,
                            df_ether_polar.u_timestamp,
                             search_keyword)

From : 10-03-2016 16:00:00
Until: 28-02-2022 17:00:00


[{'name': 'CryptoCurrency Tweet',
  'url': 'https://www.openblender.io/#/dataset/explore/5ea20b7f9516293633715928',
  'id_dataset': '5ea20b7f9516293633715928',
  'features': ['associated_tweet',
   'author',
   'author_id',
   'favorite_count',
   'hashtags',
   'id',
   'links',
   'mentions',
   're_tweeter',
   'reply_count',
   'retweet_count',
   'text',
   'time',
   'type'],
  'num_observations': 182,
  'intersection': 81.56443149616328,
  'description': 'The latest Twitter threads from CryptoCurrency (@cryptocurr_ncy). With an Ivy League pedigree Andy Clay.'},
 {'name': 'Cardano Price',
  'url': 'https://www.openblender.io/#/dataset/explore/5db1f96a9516291ccd18ad4f',
  'id_dataset': '5db1f96a9516291ccd18ad4f',
  'features': ['change',
   'high',
   'low',
   'open',
   'price',
   'timestamp',
   'volume'],
  'num_observations': 1472,
  'intersection': 69.69147005444646,
  'description': 'Daily Price of Cardano'},
 {'name': 'BTC (crypto) Fear',
  'url': 'https://www.openblender

In [49]:
blend_source = {
                'id_dataset':'5ea209c495162936348f13eb',
                'feature' : 'text'
            }

# Now, let's 'timeBlend' it to our dataset
df_bitcoin_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df_bitcoin_polar.u_timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df_bitcoin_polar = pd.concat([df_bitcoin_polar, df_bitcoin_blend.loc[:, df_bitcoin_blend.columns != 'u_timestamp']], axis = 1)
df_bitcoin_polar

Task ID: '621e792f0895fafb4a9d8d4c'.
Total estimated consumption: 9795.2 processing units.
Task confirmed. Starting download..
36.0%
71.0%
100%


,date,u_timestamp,price,target,timestamp,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days
0,01-01-2013 16:00:00,1.357056e+09,13.3,0,1357056000,0,[]
1,02-01-2013 16:00:00,1.357142e+09,13.3,0,1357142400,0,[]
2,03-01-2013 16:00:00,1.357229e+09,13.4,0,1357228800,0,[]
3,04-01-2013 16:00:00,1.357315e+09,13.5,0,1357315200,0,[]
4,05-01-2013 16:00:00,1.357402e+09,13.4,0,1357401600,0,[]
...,...,...,...,...,...,...,...
2806,10-05-2021 17:00:00,1.620666e+09,55851.0,0,1620666000,7,[happy monday crypto twitter ethereum now over...
2807,11-05-2021 17:00:00,1.620752e+09,56715.0,1,1620752400,5,[druckenmiller says within 15 years the us dol...
2808,12-05-2021 17:00:00,1.620839e+09,49725.0,0,1620838800,11,[coinbase is the most downloaded app on the ap...
2809,13-05-2021 17:00:00,1.620925e+09,49747.0,1,1620925200,9,[they also had the fossil fuels fud in 1999 be...


In [50]:
blend_source = {
                'id_dataset':'5ea209c495162936348f13eb',
                'feature' : 'text'
            }

# Now, let's 'timeBlend' it to our dataset
df_ether_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df_ether_polar.u_timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df_ether_polar = pd.concat([df_ether_polar, df_ether_blend.loc[:, df_ether_blend.columns != 'u_timestamp']], axis = 1)
df_ether_polar

Task ID: '621e79420895fafb4a9d8d4c'.
Total estimated consumption: 6640.0 processing units.
Task confirmed. Starting download..
55.00000000000001%
100%


,date,u_timestamp,price,target,timestamp,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days
0,10-03-2016 16:00:00,1.457626e+09,11.75,1,1457625600,0,[]
1,11-03-2016 16:00:00,1.457712e+09,11.95,1,1457712000,0,[]
2,12-03-2016 16:00:00,1.457798e+09,12.92,1,1457798400,0,[]
3,13-03-2016 16:00:00,1.457885e+09,15.07,1,1457884800,0,[]
4,14-03-2016 16:00:00,1.457971e+09,12.50,0,1457971200,0,[]
...,...,...,...,...,...,...,...
1820,24-02-2022 17:00:00,1.645722e+09,2596.16,1,1645722000,8,[deutsche bank conducted a survey and found cr...
1821,25-02-2022 17:00:00,1.645808e+09,2767.33,1,1645808400,14,"[, kccofficialtw, breaking bny mellon the worl..."
1822,26-02-2022 17:00:00,1.645895e+09,2776.37,1,1645894800,8,[english premier league to close a 4 year nft ...
1823,27-02-2022 17:00:00,1.645981e+09,2616.15,0,1645981200,10,[the official ukraine government account now a...


In [51]:
df_bitcoin_new_polar = df_bitcoin_polar

In [52]:
df_bitcoin_new_polar.to_csv('Bitcoinwithtext.csv', index = False)

In [53]:
df_bitcoin_new_polar

,date,u_timestamp,price,target,timestamp,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days
0,01-01-2013 16:00:00,1.357056e+09,13.3,0,1357056000,0,[]
1,02-01-2013 16:00:00,1.357142e+09,13.3,0,1357142400,0,[]
2,03-01-2013 16:00:00,1.357229e+09,13.4,0,1357228800,0,[]
3,04-01-2013 16:00:00,1.357315e+09,13.5,0,1357315200,0,[]
4,05-01-2013 16:00:00,1.357402e+09,13.4,0,1357401600,0,[]
...,...,...,...,...,...,...,...
2806,10-05-2021 17:00:00,1.620666e+09,55851.0,0,1620666000,7,[happy monday crypto twitter ethereum now over...
2807,11-05-2021 17:00:00,1.620752e+09,56715.0,1,1620752400,5,[druckenmiller says within 15 years the us dol...
2808,12-05-2021 17:00:00,1.620839e+09,49725.0,0,1620838800,11,[coinbase is the most downloaded app on the ap...
2809,13-05-2021 17:00:00,1.620925e+09,49747.0,1,1620925200,9,[they also had the fossil fuels fud in 1999 be...


In [54]:
df_ether_new_polar = df_ether_polar

In [55]:
df_ether_new_polar.to_csv('Ethereumwithtext.csv', index = False)

In [56]:
df_ether_new_polar

,date,u_timestamp,price,target,timestamp,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days
0,10-03-2016 16:00:00,1.457626e+09,11.75,1,1457625600,0,[]
1,11-03-2016 16:00:00,1.457712e+09,11.95,1,1457712000,0,[]
2,12-03-2016 16:00:00,1.457798e+09,12.92,1,1457798400,0,[]
3,13-03-2016 16:00:00,1.457885e+09,15.07,1,1457884800,0,[]
4,14-03-2016 16:00:00,1.457971e+09,12.50,0,1457971200,0,[]
...,...,...,...,...,...,...,...
1820,24-02-2022 17:00:00,1.645722e+09,2596.16,1,1645722000,8,[deutsche bank conducted a survey and found cr...
1821,25-02-2022 17:00:00,1.645808e+09,2767.33,1,1645808400,14,"[, kccofficialtw, breaking bny mellon the worl..."
1822,26-02-2022 17:00:00,1.645895e+09,2776.37,1,1645894800,8,[english premier league to close a 4 year nft ...
1823,27-02-2022 17:00:00,1.645981e+09,2616.15,0,1645981200,10,[the official ukraine government account now a...
